#### Code to download and separate geopotential height/VT/U data.

Copied on May 7, 2025. 

In [1]:
#import statements... I think these are all of the relevant ones to what I am doing here. 
import datetime as dt
import numpy as np
import xarray as xr
import netCDF4
import pickle
import xesmf as xe

In [2]:
#year = [i for i in range(1991,2022)] #indicate years for data, needed for opening files
year = [i for i in range(1959,2022)] 
#index = [i for i in range(1,43)]

In [3]:
len(year)

63

In [6]:
##check levels of V datasets 
vfile = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/V/era5_an_vwind_reg2_daily_2022.nc")
vfile['var132']

<xarray.DataArray 'var132' (time: 365, plev: 37, lat: 91, lon: 180)>
[221211900 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * plev     (plev) float64 100.0 200.0 300.0 500.0 ... 9.5e+04 9.75e+04 1e+05
  * time     (time) datetime64[ns] 2022-01-01T09:00:00 ... 2022-12-31T09:00:00
Attributes:
    table:    128

In [7]:
##check levels of V datasets 
ufile = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/U/era5_an_uwind_reg2_daily_1960.nc")
ufile['var131']

<xarray.DataArray 'var131' (time: 366, plev: 37, lat: 91, lon: 180)>
[221817960 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * plev     (plev) float64 100.0 200.0 300.0 500.0 ... 9.5e+04 9.75e+04 1e+05
  * time     (time) datetime64[ns] 1960-01-01T09:00:00 ... 1960-12-31T09:00:00
Attributes:
    table:         128
    cell_methods:  time: mean

In [9]:
##check levels of T datasets 
tfile = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/T/era5_an_temp_reg2_daily_1959.nc")
tfile['var130']

<xarray.DataArray 'var130' (time: 365, plev: 37, lat: 91, lon: 180)>
[221211900 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * plev     (plev) float64 100.0 200.0 300.0 500.0 ... 9.5e+04 9.75e+04 1e+05
  * time     (time) datetime64[ns] 1959-01-01T09:00:00 ... 1959-12-31T09:00:00
Attributes:
    table:         128
    cell_methods:  time: mean

In [10]:
##check levels of GPH datasets 
gfile = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/GPH/era5_an_geopot_reg2_daily_1959.nc")
gfile['var129']

<xarray.DataArray 'var129' (time: 365, plev: 37, lat: 91, lon: 180)>
[221211900 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 90.0 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0 -90.0
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * plev     (plev) float64 100.0 200.0 300.0 500.0 ... 9.5e+04 9.75e+04 1e+05
  * time     (time) datetime64[ns] 1959-01-01T09:00:00 ... 1959-12-31T09:00:00
Attributes:
    table:         128
    cell_methods:  time: mean

#### Now I am able to see that these all possess the same number of levels so ... 

For U, I will need to have an array (len(year), 183, 37, 36, 180), U wind betweem 20 and 90.  

For V and T, I will need to reshape for (len(year), 183, 37, 13, 180). 

For GPH, this will be (len(year), 183, 37, 13, 180)

In [11]:
print(len(np.arange(75,42.5,-2.5)))
print(len(np.arange(90,58.5,-2.5)))
print(len(np.arange(90,18,-2)))
print(len(np.arange(0, 360.5, 2.5)))

13
13
36
145


#### SAVE U WIND.

In [12]:
#create empty arrays for the years of data
value = np.empty((len(year),183,37,36,180))
value[:] = np.nan

In [13]:
##U WIND FILE
#this is my attempt at looping through and combining the files lol WITH REGRID
for i in range(len(year)):
    print(year[i])
    #start by designating leap years because those arrays will be of a different size
    ##1959:
    if year[i] == 1959 or year[i] == 1963 or year[i] == 1967 or year[i] == 1971 or year[i] == 1975 or year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
        
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/U/era5_an_uwind_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var131"] 
        g_data1 = g_files1.loc[dict(lat=slice(90,20))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/U/era5_an_uwind_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var131"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(90,20))]
        values2 =  g_data2.values


        #combine into np array
        value[i,:92,:,:,:] =  values1[273:,:,:,:]
        value[i,92:,:,:,:] =  values2[:91,:,:,:] 
        continue
        
    #this loop is because of the weird indexing in years of leap years
    ###1959
    if  year[i] == 1960 or  year[i] == 1964 or  year[i] == 1968 or  year[i] == 1972 or  year[i] == 1976 or year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/U/era5_an_uwind_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var131"] 
        g_data1 = g_files1.loc[dict(lat=slice(90,20))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/U/era5_an_uwind_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var131"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(90,20))]
        values2 =  g_data2.values

        #combine into np array
        value[i,:92,:,:,:] = values1[274:,:,:,:]
        value[i,92:151,:,:,:] = values2[:59,:,:,:]
        value[i,152:,:,:,:] = values2[60:91,:,:,:]
        continue
        
    #everything else and repeat 
    else:
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/U/era5_an_uwind_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var131"] 
        g_data1 = g_files1.loc[dict(lat=slice(90,20))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/U/era5_an_uwind_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var131"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(90,20))]
        values2 =  g_data2.values

        #combine into np array
        value[i,:92,:,:,:] = values1[273:,:,:,:]
        value[i,92:151,:,:,:] = values2[:59,:,:,:]
        value[i,152:,:,:,:] = values2[60:91,:,:,:]
        continue

1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [14]:
value[1,140:,0,0,0]

array([-16.47560501, -22.0357132 , -23.2856102 , -19.18064117,
       -21.02908707, -18.93673706, -13.61249161, -10.7073822 ,
       -17.81741333, -26.35971832, -34.50760651,          nan,
       -43.50793839, -47.01453018, -45.01369858, -51.62160492,
       -48.39819717, -54.11872482, -52.13436127, -50.93945312,
       -49.44746399, -46.73948669, -39.43329239, -23.41979599,
       -28.14545441, -15.11882019,  -4.4665184 ,   7.44122314,
         9.05577469,   1.76433182,   2.3951416 ,  -0.74024582,
        -8.52905655,  -9.62877655,  -8.10884476,  -5.69981003,
        -2.0334549 ,   3.6619072 ,   3.53544593,   2.07299662,
         3.06555939,   3.23908567,   4.88388443])

In [15]:
pickle.dump(value, open("./large_area_data/1959_u_2deg.p", 'wb')) 
##gph will need to be divided by 10. 

#### SAVE V WIND.

In [16]:
#create empty arrays for the years of data
value = np.empty((len(year),183,37,15,180))
value[:] = np.nan

In [17]:
##V WIND FILE
#this is my attempt at looping through and combining the files lol WITH REGRID
for i in range(len(year)):
    print(year[i])
    #start by designating leap years because those arrays will be of a different size
    ##1959:
    if year[i] == 1959 or year[i] == 1963 or year[i] == 1967 or year[i] == 1971 or year[i] == 1975 or year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
        
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/V/era5_an_vwind_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var132"] 
        g_data1 = g_files1.loc[dict(lat=slice(75,45))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/V/era5_an_vwind_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var132"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(75,45))]
        values2 =  g_data2.values


        #combine into np array
        value[i,:92,:,:,:] =  values1[273:,:,:,:]
        value[i,92:,:,:,:] =  values2[:91,:,:,:] 
        continue
        
    #this loop is because of the weird indexing in years of leap years
    ###1959
    if  year[i] == 1960 or  year[i] == 1964 or  year[i] == 1968 or  year[i] == 1972 or  year[i] == 1976 or year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/V/era5_an_vwind_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var132"] 
        g_data1 = g_files1.loc[dict(lat=slice(75,45))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/V/era5_an_vwind_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var132"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(75,45))]
        values2 =  g_data2.values

        #combine into np array
        value[i,:92,:,:,:] = values1[274:,:,:,:]
        value[i,92:151,:,:,:] = values2[:59,:,:,:]
        value[i,152:,:,:,:] = values2[60:91,:,:,:]
        continue
        
    #everything else and repeat 
    else:
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/V/era5_an_vwind_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var132"] 
        g_data1 = g_files1.loc[dict(lat=slice(75,45))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/V/era5_an_vwind_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var132"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(75,45))]
        values2 =  g_data2.values

        #combine into np array
        value[i,:92,:,:,:] = values1[273:,:,:,:]
        value[i,92:151,:,:,:] = values2[:59,:,:,:]
        value[i,152:,:,:,:] = values2[60:91,:,:,:]
        continue

1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [18]:
value[1,140:,0,0,0]

array([-10.44225693,  -5.8478241 ,  -9.44345093,  -7.1171608 ,
        -5.50409698,  -2.4201355 ,  -3.25915527,   9.42077637,
        -1.27486801,  -9.86222839, -10.04367447,          nan,
       -21.36532593, -24.51795959, -35.84249115, -38.18339157,
       -35.0646019 , -36.95631027, -33.78686142, -35.54520416,
       -38.60417557, -36.27721024, -27.48869324, -27.71282196,
       -22.32458496, -11.1106987 ,  -8.59675598,  -0.6574707 ,
        -1.97827148,  -3.20763397,  -2.5712204 ,  -1.25096512,
        -2.40694046,  -1.65813446,  -2.74965286,   1.41397476,
         6.6808548 ,   5.82423019,   3.4754982 ,   0.96657562,
        -3.7050209 ,  -4.05934143,  -0.20933151])

In [19]:
pickle.dump(value, open("./large_area_data/1959_v_2deg.p", 'wb')) 
##gph will need to be divided by 10. 

#### SAVE TEMPS.

In [20]:
#create empty arrays for the years of data
value = np.empty((len(year),183,37,15,180))
value[:] = np.nan

In [21]:
##T FILE
#this is my attempt at looping through and combining the files lol WITH REGRID
for i in range(len(year)):
    print(year[i])
    #start by designating leap years because those arrays will be of a different size
    ##1959:
    if year[i] == 1959 or year[i] == 1963 or year[i] == 1967 or year[i] == 1971 or year[i] == 1975 or year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
        
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/T/era5_an_temp_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var130"] 
        g_data1 = g_files1.loc[dict(lat=slice(75,45))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/T/era5_an_temp_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var130"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(75,45))]
        values2 =  g_data2.values


        #combine into np array
        value[i,:92,:,:,:] =  values1[273:,:,:,:]
        value[i,92:,:,:,:] =  values2[:91,:,:,:] 
        continue
        
    #this loop is because of the weird indexing in years of leap years
    ###1959
    if  year[i] == 1960 or  year[i] == 1964 or  year[i] == 1968 or  year[i] == 1972 or  year[i] == 1976 or year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/T/era5_an_temp_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var130"] 
        g_data1 = g_files1.loc[dict(lat=slice(75,45))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/T/era5_an_temp_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var130"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(75,45))]
        values2 =  g_data2.values

        #combine into np array
        value[i,:92,:,:,:] = values1[274:,:,:,:]
        value[i,92:151,:,:,:] = values2[:59,:,:,:]
        value[i,152:,:,:,:] = values2[60:91,:,:,:]
        continue
        
    #everything else and repeat 
    else:
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/T/era5_an_temp_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var130"] 
        g_data1 = g_files1.loc[dict(lat=slice(75,45))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/T/era5_an_temp_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var130"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(75,45))]
        values2 =  g_data2.values

        #combine into np array
        value[i,:92,:,:,:] = values1[273:,:,:,:]
        value[i,92:151,:,:,:] = values2[:59,:,:,:]
        value[i,152:,:,:,:] = values2[60:91,:,:,:]
        continue

1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


FileNotFoundError: [Errno 2] No such file or directory: b'/network/rit/lab/wulab/Datasets/ERA5/daily/T/era5_an_temp_reg2_daily_2022.nc'

In [25]:
value[62,90:130,0,0,0]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan])

In [26]:
pickle.dump(value, open("./large_area_data/1959_t_2deg.p", 'wb')) 
##gph will need to be divided by 10. 

#### SAVE GPH. 

I need to fix this to be just N of 20. lol. 

In [27]:
#create empty arrays for the years of data
value = np.empty((len(year),183,37,36,180))
value[:] = np.nan

In [28]:
##GPH FILE
#this is my attempt at looping through and combining the files lol WITH REGRID
for i in range(len(year)):
    print(year[i])
    #start by designating leap years because those arrays will be of a different size
    ##1959:
    if year[i] == 1959 or year[i] == 1963 or year[i] == 1967 or year[i] == 1971 or year[i] == 1975 or year[i] == 1979 or year[i] == 1983 or year[i] == 1987 or year[i] == 1991 or year[i] == 1995 or year[i] == 1999 or year[i] == 2003 or year[i] == 2007 or year[i] == 2011 or year[i] == 2015 or year[i] == 2019:
        
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/GPH/era5_an_geopot_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var129"] 
        g_data1 = g_files1.loc[dict(lat=slice(90,20))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/GPH/era5_an_geopot_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var129"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(90,20))]
        values2 =  g_data2.values


        #combine into np array
        value[i,:92,:,:,:] =  values1[273:,:,:,:]
        value[i,92:,:,:,:] =  values2[:91,:,:,:] 
        continue
        
    #this loop is because of the weird indexing in years of leap years
    ###1959
    if  year[i] == 1960 or  year[i] == 1964 or  year[i] == 1968 or  year[i] == 1972 or  year[i] == 1976 or year[i] == 1980 or year[i] == 1984 or year[i] == 1988 or year[i] == 1992 or year[i] == 1996 or year[i] == 2000 or year[i] == 2004 or year[i] == 2008 or year[i] == 2012 or year[i] == 2016 or year[i] == 2020:
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/GPH/era5_an_geopot_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var129"] 
        g_data1 = g_files1.loc[dict(lat=slice(90,20))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/GPH/era5_an_geopot_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var129"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(90,20))]
        values2 =  g_data2.values

        #combine into np array
        value[i,:92,:,:,:] = values1[274:,:,:,:]
        value[i,92:151,:,:,:] = values2[:59,:,:,:]
        value[i,152:,:,:,:] = values2[60:91,:,:,:]
        continue
        
    #everything else and repeat 
    else:
        gfile1 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/GPH/era5_an_geopot_reg2_daily_"+str(year[i])+".nc")
        g_files1 = gfile1["var129"] 
        g_data1 = g_files1.loc[dict(lat=slice(90,20))]
        values1 =  g_data1.values
        
        #file 2 for end
        gfile2 = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/daily/GPH/era5_an_geopot_reg2_daily_"+str(year[i]+1)+".nc")
        g_files2 = gfile2["var129"] #next line will reduce this to the mean 
        g_data2 = g_files2.loc[dict(lat=slice(90,20))]
        values2 =  g_data2.values

        #combine into np array
        value[i,:92,:,:,:] = values1[273:,:,:,:]
        value[i,92:151,:,:,:] = values2[:59,:,:,:]
        value[i,152:,:,:,:] = values2[60:91,:,:,:]
        continue

1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [31]:
value[1,140:,0,0,0]

array([44159.10104485, 44145.28860856, 44135.66513761, 44117.30377166,
       44227.97846585, 44235.60779817, 44414.51643731, 44532.58154944,
       44721.42584098, 44773.09505607, 44994.26605505,            nan,
       45213.53529562, 45408.85894495, 45565.36697248, 45607.64525994,
       45850.91743119, 46036.16207951, 46388.21992864, 46537.68794597,
       46632.47324159, 46555.67023445, 46540.04841998, 46484.72859327,
       46483.87168705, 46721.54689093, 47040.94036697, 47179.59989806,
       47088.76465341, 46976.43667176, 46877.00688073, 46849.7833843 ,
       46844.78848114, 46779.38328236, 46770.14525994, 46801.00025484,
       46810.03440367, 46807.77268094, 46784.87512742, 46786.28950051,
       46757.75356779, 46768.56842508, 46796.35894495])

In [30]:
value = value/9.81

In [32]:
pickle.dump(value, open("./large_area_data/1959_gph_2deg.p", 'wb')) 
##gph will need to be divided by 10. 